In [ ]:
import numpy as np
import cv2
import pathlib
import os 

import tensorflow as tf

print(tf.__version__)
print(tf.keras.__version__)

import matplotlib.pyplot as plt

#data obtained from https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia/data

In [ ]:
train_dir = "./Pneumonia_dataset/chest_xray/train"
test_dir = "./Pneumonia_dataset/chest_xray/test"
val_dir = "./Pneumonia_dataset/chest_xray/val"

## TRY 1

In [ ]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rescale=1./255)

train_iter = datagen.flow_from_directory(train_dir, 
                                        batch_size = 8, 
                                        target_size = (1024,1024))
val_iter = datagen.flow_from_directory(test_dir, 
                                        batch_size=8, 
                                        target_size=(1024, 1024))
test_iter = datagen.flow_from_directory(val_dir, 
                                        batch_size=8, 
                                        target_size=(1024, 1024))

train_images, train_labels = next(train_iter)
test_images, test_labels = next(test_iter)
print('Batch shape=%s, min=%.3f, max=%.3f' % (train_images.shape, train_images.min(), train_images.max()))

In [ ]:
def build_cnn():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', strides=(2,2), input_shape=(1024, 1024, 3)))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(32, (3, 3), strides=(2,2),activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D((2, 2)))
    model.add(tf.keras.layers.Conv2D(16, (3, 3), strides=(2,2),activation='relu'))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(16, activation='relu'))
    model.add(tf.keras.layers.Dense(2, activation='softmax'))
    return model

model = build_cnn()

In [ ]:
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_iter, 
                    epochs=100,
                    validation_data=test_iter, 
                    validation_steps=32)

In [ ]:
print(history.history.keys())

plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy') 
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')

plt.xlabel('Epoch')
plt.ylabel('Metrics')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')
plt.show()

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

print(test_acc)

## TRY 2

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop

In [ ]:
train_datagen = ImageDataGenerator(
                        rescale=1./255,
                        rotation_range=40,
                        width_shift_range=0.2,
                        height_shift_range=0.2,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1.0/255.)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        batch_size=20,
        class_mode='binary',
        target_size=(1024, 1024))

validation_generator = test_datagen.flow_from_directory(
        val_dir,
        batch_size=20,
        class_mode='binary',
        target_size=(1024, 1024))

model = tf.keras.models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=(1024, 1024, 3)),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.MaxPooling2D(2,2),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
                optimizer=RMSprop(learning_rate=0.001),
                metrics=['accuracy'])

In [ ]:
history = model.fit(
        train_generator,
        validation_data=validation_generator,
        steps_per_epoch=100,
        epochs=20,
        validation_steps=50,
        verbose=3)

In [ ]:
classes = ["NORMAL", "PNEUMONIA"]
image_dimensions = [1024, 1024]

# Predict the class of the input image from the loaded model
for file in os.listdir(val_dir):
    image_dir = os.path.join(val_dir, file)
    if os.path.isdir(image_dir):
        for img in os.listdir(image_dir):
            image_path = os.path.join(image_dir, img)
            image = tf.io.read_file(image_path)
            image = tf.io.decode_image(image, channels=3)
            image = tf.image.resize(image, image_dimensions)
            image = image / 255.0  # Normalize pixel values between 0 and 1
            image = tf.expand_dims(image, axis=0)  # Add batch dimension
            predicted = model(image)
            max_probability = np.max(predicted)
            max_index = np.argmax(predicted)
            print("Image:", image_path, " Predicted", classes[max_index], " Actual:", file, " Probability: ", max_probability)